In [2]:
import tensorflow as tf
import numpy as np
import importlib 
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [13]:
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

In [2]:
def rnn_model(features, labels, mode, params):
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  input_layer = features["x"]    
  cell1 = tf.nn.rnn_cell.LSTMCell(name = 'cell1', num_units = params['hidden_layers'][0])
  outputs, state = tf.nn.dynamic_rnn(cell=cell1,
                                   inputs=input_layer,
                                   dtype=tf.float64)
  outputs = tf.nn.relu(outputs)      
  outputs = tf.layers.dropout(inputs=outputs, rate=0.5, training=(mode==tf.estimator.ModeKeys.TRAIN))
  
  for i in range(num_hidden_layers-1):  
      cell = tf.nn.rnn_cell.LSTMCell(name = 'cell'+str(i+2),num_units = params['hidden_layers'][i+1])
      outputs, state = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
      outputs = tf.nn.relu(outputs)
  
  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  tf.summary.scalar('loss', loss)
      

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                      sess.graph)
  test_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/test')
  

In [19]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=64,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = eeg_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D302FFF2B0>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /model/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 11 into /model/model.ckpt.
INFO:t

In [36]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = eeg_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D3359F0E80>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.25427197 0.25029955 0.24535519 0.2

In [39]:
X, y = import_data(every=True)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D3026FEC88>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.23812319 0.26113917 0.25401754 0.2

In [41]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D334348940>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.24662683 0.25936754 0.24301177 0.2

In [44]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [64, 64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D3026FD4A8>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.26066477 0.23990037 0.24609502 0.2

In [45]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D3346299E8>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.24685718 0.26268634 0.2488772  0.2

In [46]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D3026A2DA0>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.25175045 0.25311701 0.25030936 0.2

In [49]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D33608D470>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.2284479  0.24698179 0.25529095 0.2

In [50]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_subject(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [64,64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D30242CB70>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.2480113  0.272019   0.2293732  0.2

In [52]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_subject(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32,32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': '/model/', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D30205F6D8>, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /model/model.ckpt.
INFO:tensorflow:probabilities = [[0.2916902  0.21654832 0.27040909 0.2

In [3]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_subject(X, y,train_all=False)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, 
                                        params = {'hidden_layers' : [64,64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_tf_random_seed': None, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E32BF92A58>, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_model_dir': 'C:\\Users\\Sajeed\\AppData\\Local\\Temp\\tmph95k2kn8', '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Sajeed\AppData\Local\Temp\tmph95k2k

In [5]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y,standardize=False)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, 
                                        params = {'hidden_layers' : [64,64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_tf_random_seed': None, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E302FB6710>, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_model_dir': 'C:\\Users\\Sajeed\\AppData\\Local\\Temp\\tmp12953bz4', '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Sajeed\AppData\Local\Temp\tmp12953b

In [7]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, 
                                        params = {'hidden_layers' : [32,32,32,32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_tf_random_seed': None, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E32BF73C50>, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_model_dir': 'C:\\Users\\Sajeed\\AppData\\Local\\Temp\\tmp0ntc5v_c', '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Sajeed\AppData\Local\Temp\tmp0ntc5v

In [9]:
X, y = import_data(every=False)
X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, 
                                        params = {'hidden_layers' : [64], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_tf_random_seed': None, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E319027D68>, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_model_dir': 'C:\\Users\\Sajeed\\AppData\\Local\\Temp\\tmpqck_j2a_', '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Sajeed\AppData\Local\Temp\tmpqck_j2